# <font color = firebrick>Tutorial 6: Web Scraping<a id='home'></a>

Economists work primarily with ready-made datasets. However, many economic research questions require unique data that may not be available in a pre-packaged dataset. The internet provides a wealth of such data if we know how to access it, and *web scraping* is the process of collecting this data into a format suitable for analysis, like a DataFrame. This approach can also help reverse-engineer data from a website, allowing us to construct custom datasets.

Web scraping exists in a legal [gray area](https://en.wikipedia.org/wiki/Web_scraping#United_States). Many websites discourage scraping, as data is often a valuable asset for businesses, and indiscriminate scraping can cause disruptions. In this tutorial, we’ll discuss how to check if a website permits scraping and will practice on websites that allow it.

To clarify—[web scraping is not illegal](https://techcrunch.com/2022/04/18/web-scraping-legal-court/#:~:text=In%20its%20second%20ruling%20on,computer%20hacking%20under%20U.S.%20law.). As long as data is publicly available, it is legally accessible, though always within limits and ethical considerations.

Here are some ways to approach web scraping:

1. Copying and pasting data manually from a webpage to a file. This is simple but time-consuming for large datasets.

2. Using software tools like browser extensions to automate scraping. While this can be convenient, it has limitations in terms of flexibility and precision.

3. Writing code yourself, which provides more control and versatility. This will be our approach, using Python libraries.

Here's an outline of today's tutorial:

1. [Checking website permissions.](#permissions)
2. [HTML basics.](#html)
3. [Retrieving data from a webpage.](#scrape)
4. [Relevant Python packages.](#packages)


# 1. Checking website permissions [(home)](#home)<a id='permissions'></a>

Web crawlers, like those used by search engines, only index a portion of the internet—the surface web. Websites can choose to restrict access to certain parts of their content by using a `robots.txt` file. This file, located at `http://www.website.com/robots.txt`, tells search engine crawlers which pages they may or may not access. Respecting a site's `robots.txt` directives is crucial to avoid legal issues and maintain ethical standards.

The `robots.txt` file can contain various instructions:

1. Full Access
> User-agent: * <br>
> Disallow:

This allows bots to crawl any part of the site. A website with this setting has no restrictions on crawling.

2. No Access

> User-agent: * <br>
> Disallow: /

When a `robots.txt` file contains this directive, it means the entire site is off-limits to crawlers. It’s best to avoid scraping such websites to stay within legal bounds.

3. Partial Access

> User-agent: * <br>
> Disallow: /folder/ <br>
> Disallow: /file.html

Some sites only restrict certain pages or sections. It’s important to respect these restrictions when scraping.

4. Crawl Rate Limit

> Crawl-delay: 11

This limits the rate at which a crawler can request pages to avoid overloading the server. A crawl delay of 11 seconds, for instance, means that bots must wait 11 seconds between requests.

5. Restricted Visit Times

> Visit-time: 0400-0845

Some sites allow crawling only during specified hours to reduce server load during peak times.

6. Request Rate

> Request-rate: 1/10

This setting indicates that only one page can be requested every 10 seconds, reducing strain on the server.

Throughout this tutorial, we’ll focus on scraping data from websites that explicitly allow it, keeping our approach ethical and respectful.

### Example: Delta Airlines

Visit [www.delta.com/robots.txt](https://www.delta.com/robots.txt) to see how Delta manages crawler access. You’ll notice that some pages are open to crawlers, while others—such as flight search results—are restricted to prevent bots from scraping data on availability or pricing.

> Disallow: /flight-search/book-a-flight?cacheKeySuffix= <br>
> Disallow: /flight-search/search?&tripType= <br>
> Disallow: /flight-search/search?action=


# 2. HTML basics [(home)](#home)<a id='html'></a>

Scraping is part code and part detective work. We need to look at the code that underlies a webpage to understand how it is structured. Then we can figure out how to scrape it.

Each webpage that you view in your browser is actually structured in HyperText Markup Language (HTML). HTML code has two parts:

1. the *head* which includes the title and any imports for styling and JavaScript, and 
2. the *body* which includes the content that gets displayed as a webpage. 

We’ll be interested in the body of the webpage. HTML code is comprised of tags, where a tag is described by an opening `<` and closing `\>` angular bracket with the name of the tag between; e.g., `<div></div>`, `<p>Some text</p>` etc.

The useful tags for us will be:

`<div>`: This tag groups together elements into a single entity. This tag can act as the parent for a lot of different elements so style changes which are applied here will also reflect in child elements.

`<a>`: URL links are described in this tag. The webpage that will get loaded when the link is clicked on is given in its property href.

`<p>`: Used when information is displayed on the webpage as a block of text ($\approx$ "paragraph").

`<span>`: This tag is used when information is to be displayed inline. Moreover, when two such tags are placed side by side, they'll appear in the same line unlike the paragraph tag.

`<table>`: Tables are displayed in HTML with the help of this tag; i.e., data is displayed in cells formed by intersection of rows and columns.

### Detective work

Every website has a different structure and there are a few important things to think about when building a web scraper:

* What is the structure of the data contained on the page?
* How do we get to those web pages?
* Will you need to gather more data from the next page?
* Is the structure repeatable?

# 3. Retrieving data [(home)](#home)<a id='scrape'></a>

We will scrape a webpage built specifically for practicing scraping: [toscrape.com](http://toscrape.com/). We will scrape the fake [bookstore page](http://books.toscrape.com/catalogue/category/books_1/index.html). 

Go to the site. Right click on part of the page and choose "inspect" if you are using the Chrome browser. This will open a new window pane that displays the HTML underneath the page. 

Inspect different parts of the site and try to find the parts that list a single book's information. We will need to find the tags that contain the information that we want. 

This can take some time...

# <font color='red'>Practice</font>

Our goal is to scrape the prices, titles, and ratings. 

1. Use the inspector to find the `article class` that contains the book: "A Light in the Attic." What is the class name?
2. Within the `article class`, find the book's title. What html element contains the title?
2. Within the `article class`, find the book's price. What html element contains the title?
2. Within the `article class`, find the book's rating. What html element contains the rating?


# 4. Packages [(return)](#home)<a id='packages'></a>
We need some new packages: 

- `requests`: We use this library to open the url from which we would like to extract the data. This is a standard library included with your python distribution so you won't have to install it.


- `BeautifulSoup`: This library helps us to get the HTML structure of the page that we want to work with. We can then, use its functions to access specific elements and extract relevant information. Anaconda includes this package.

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

This code takes a url as input, opens the url using the `.get()` method, and parses (i.e., breaks up) the corresponding HTML files into a usable data type.

We saw this package when we retrieved data from the Census API, too. 

In [ ]:
url = 'http://books.toscrape.com/catalogue/category/books_1/index.html'
    
results = requests.get(url)  

# Make the content we grabbed easy to read by using BeautifulSoup.
content = BeautifulSoup(results.text, 'html.parser')
type(content)

So now we have a BeautifulSoup object that holds the parsed HTML. We can use the `.prettify()` method to take a look. 

In [ ]:
print(content.prettify())

## Get all the books on the page

Each book's information lives in a an article tag of the form

```html
<article class="product_pod">
```

We use the `.find_all()` method and tell it to find all the `article` tags with `class` of `product_pod`. We pass `find_all()` the type of element to look for and a dictionary of attribute values.

In [ ]:
books = content.find_all('article', {'class' : 'product_pod'})
type(books)

The ResultSet object is iterable. We will be able to loop over all the books in this object and extract the data we need. 

In [ ]:
# The first article with class="product_pod".
books[0]

### Get the title of the book

After studying the HTML code above, I see that the book title is contained in the link (an `a` tag) in the third-level header (the `h3` tag). 

We use the `.` notation to retrieve the first instance of the `h3` tag (there is only one instance). Think of the tags nested within the `books[0]` tag as attributes of `books[0]`. As usual in python, we use the `.` to access attributes. 

We use the square brackets to references a *key* within a tag&mdash;like we would do with a dict.

So the code 

```python 
books[0].h3.a['title']
```

asks for the first `h3` tag in `books[0]` and the first `a` tag within the `h3` tag. From the `a` tag, we ask for the value associated with the key 'title'>

In [ ]:
books[0].h3.a

In [ ]:
books[0].h3.a['title']

### Get the price of the book

There is more than one `div` tag here, so we use `.find()` to find the one with class 'product_price'. Inside of this `div`, the first `p` tag has the price (although the `p` is class product_color...) so we use the `.` to grab it.

Within the `p` tag, we need the text attribute. 

In [ ]:
# This works, too. It is more robust to a change in the order of the `p` tags.
# print(books[0].find('div', class_='product_price').find('p', class_='price_color').text)

books[0].find('div', {'class' : 'product_price'}).p.text


We need to take care of the non-numeric data. We will take care of that once we have scraped all the data. 

### Get the book's rating

The star rating is encoded in the `p` tag with a class of 'star-rating X' where X could be Zero, One, Two, etc...

In [ ]:
# Find the `p` that contain the string 'star-rating'.
x = books[0].find('p', {'class':'star-rating'})
x['class'][1]

## Create the DataFrame

1. Loop over the books and extract the information we want. Store the information in lists.
2. Convert the set of list to a DataFrame.


In [ ]:
titles, prices, stars = [], [], []

for book in books:
    titles.append(book.h3.a['title'])
    prices.append(book.find('div', {'class' : 'product_price'}).p.text)
    stars.append(book.find('p', {'class' : 'star-rating'})['class'][1])

In [ ]:
print(titles[19], prices[19], stars[19], sep = '; ')

In [ ]:
books_df = pd.DataFrame({
    'title': titles,
    'price': prices,
    'rating': stars,
})

books_df.head()

### Data cleanup

I see two issues. The non-numeric characters in 'price' and in rating. Let's fix them. 

Let's start by using a regex to clean up the price data.

In [ ]:
# Option 1: We could also use regular expressions to extract the string
books_df['price'] = books_df['price'].str.extract(r'(\d+.\d+)').astype(float) # Last part converts to a float.

# Option 2: Slicing
# books_df['price'] = books_df['price'].str.slice(2,).astype(float)

In [ ]:
books_df.dtypes

Let's make the ratings numeric. I wonder is there is a python library somewhere that reads alphabetic numbers and converts them to integers? Seems plausible. Since there are only six potential values, the brute force method isn't too costly here.

In [ ]:
convert = {'Zero':0, 'One':1, 'Two':2, 'Three':3, 'Four':4, 'Five':5}
books_df['rating'] = books_df['rating'].replace(convert).astype(int)

In [ ]:
print(books_df.head(), '\n')
print(books_df.dtypes)



# Retrieving all the books

We can extend the above code to grab more books by leveraging the pattern the url. Click on the 'next' button at the bottom of the Books to Scrape webpage. Look at the url. It reads 

```
http://books.toscrape.com/catalogue/category/books_1/page-2.html
```

Each additional page increments the 'page-x' part of the url. We simply need to loop over all the pages and scrape away.


### Don't be a jerk!

Since we'll be making several calls to the Books to Scrape server, we want to be respectful and insert a pause between calls. Failing to do this would result in your computer generating a series of quick calls to the host server. If the server is not clever, it will respond to each call and your scraper will monopolize the server, limiting others' ability to access the information. This is the basis for a [Denial of Service (DoS)](https://en.wikipedia.org/wiki/Denial-of-service_attack) attack. If the server is clever, it will block your IP and your scrape will end. 

We'll use the following packages to delay and randomize the url calls:

```python
from time import sleep
from random import randint
```

Specifically, before each url call in the loop, we'll pause the program for a random period of time; i.e., 

```python
sleep(randint(2,10))
```

The randomness is to mimic human behavior in the event the host web server is looking for bots.

In [ ]:
from time import sleep
from random import randint

In [ ]:
titles, prices, stars = [], [], []

for page in range(1, 51):
    
    # Pause for a random time between 2 and 10 seconds. Look less like a bot.
    print('waiting...', end='')
    sleep(randint(2,10))
    
    # Get the contents of the webpage.
    print('scraping page', page)
    url = 'http://books.toscrape.com/catalogue/category/books_1/page-' + str(page) + '.html'
    content = BeautifulSoup(requests.get(url).text, 'html.parser')
    books = content.find_all('article', class_='product_pod')
    
    # Extract the data we need.
    for book in books:
        titles.append(book.h3.a['title'])
        prices.append(book.find('div', {'class':'product_price'}).p.text)
        stars.append(book.find('p', {'class':'star-rating'})['class'][1])

        
# Create the DataFrame and clean it up.
books_df = pd.DataFrame({'title': titles, 'price': prices, 'rating': stars})
books_df['price'] = books_df['price'].str.slice(2,).astype(float)
convert = {'Zero':0, 'One':1, 'Two':2, 'Three':3, 'Four':4, 'Five':5}
books_df['rating'] = books_df['rating'].replace(convert).astype(int)

In [ ]:
books_df.shape

In [ ]:
books_df.tail()

# <font color='red'>Practice</font>

http://books.toscrape.com/ has a directory of book genres in the left-hand sidebar. Scrape the the topics list and create a pandas Series that contains the topics.
HTML jargon: `ul` is 'unordered list' and `li` is 'list item'. See how much easier markdown is to write?

Steps: 

1. Get the page content. Any page will do, since they all have the sidebar. I scraped page 1 again.

2. Get the `ul` with the list of headings. 

3. Use `find_all()` to get all the list items (`li`). 

4. From each list item, extract the genre text. Store them in a a list.  

5. Turn the list into a pandas Series.

6. Clean up the test. Try `str.strip()` to remove the whitespace. 

## If you have time (and interest) ...

This one is a bit harder because you need to traverse a table but you can defnitely do it.

Go to http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html. It is the product page for a particular book. 

From this page, scrape and print out the UPC and the number of reviews. Print them out.

HTML jargon: `tr` is 'table row'.

Steps: 

1. Get the page content.

2. Get all the table rows (`tr`). 

3. Check the table rows for the ones that contain "UPC" and "Number of Reviews." Extract the data and print it out.